In [15]:
import comet_ml
from comet_ml import Experiment

In [16]:
import torch_optimizer as opt

In [17]:
import seaborn as sns
import pandas as pd
import time
import cv2
import os
import albumentations
import timm
from pytorch_lightning.loggers import CometLogger
import pickle
from sklearn.model_selection import StratifiedKFold
from pytorch_lightning.callbacks import ModelCheckpoint,EarlyStopping
from torchvision.models import resnet50,resnet18
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from albumentations.pytorch import ToTensorV2
import json
import torch.nn.functional as F
import numpy as np
import random
import pytorch_lightning as pl
import torch.optim as optim
import torch.nn as nn

from torch.utils.data import Dataset,DataLoader
import torch
from sklearn.utils.class_weight import compute_class_weight

In [18]:
class cfg:
    img_size = 512
    model_name = "resnet50"
    max_epochs = 100
    batch_size = 32
    classes = np.array(['complex','frog_eye_leaf_spot','powdery_mildew','rust','scab'])
    train_root_dir = "preprocessed"
    val_root_dir = "train_images"
    patience = [5,2]
    factor= .1
    folds=5
    min_lr=1e-8


In [19]:
def seeding(seed=2021):
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.random.manual_seed(seed)
    random.seed(seed)
seeding()

In [20]:
@torch.jit.script
def mish(input):
    return input * F.tanh(F.softplus(input))

In [29]:
class Mish(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self,input):
        return mish(input)

In [30]:
def train_augs():
    return albumentations.Compose([
       albumentations.RandomResizedCrop(cfg.img_size, cfg.img_size, scale=(0.9, 1), p=1), 
       albumentations.HorizontalFlip(p=0.5),
       albumentations.VerticalFlip(p=0.5),
       albumentations.ShiftScaleRotate(p=0.5),
       albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=10, val_shift_limit=10, p=0.7),
       albumentations.RandomBrightnessContrast(brightness_limit=(-0.2,0.2), contrast_limit=(-0.2, 0.2), p=0.7),
       albumentations.CLAHE(clip_limit=(1,4), p=0.5),
       albumentations.OneOf([
           albumentations.OpticalDistortion(distort_limit=1.0),
           albumentations.GridDistortion(num_steps=5, distort_limit=1.),
           albumentations.ElasticTransform(alpha=3),
       ], p=0.2),
       albumentations.OneOf([
           albumentations.GaussNoise(var_limit=[10, 50]),
           albumentations.GaussianBlur(),
           albumentations.MotionBlur(),
           albumentations.MedianBlur(),
       ], p=0.2),
      albumentations.Resize(cfg.img_size, cfg.img_size),
      albumentations.OneOf([
          albumentations.JpegCompression(),
          albumentations.Downscale(scale_min=0.1, scale_max=0.15),
      ], p=0.2),
      albumentations.IAAPiecewiseAffine(p=0.2),
      albumentations.IAASharpen(p=0.2),
      albumentations.Cutout(max_h_size=int(cfg.img_size * 0.1), max_w_size=int(cfg.img_size * 0.1), num_holes=5, p=0.5),
      albumentations.Normalize(p=1.0),
      ToTensorV2()
    ])
    

In [31]:

def val_augs():
    return albumentations.Compose([
        albumentations.Resize(cfg.img_size,cfg.img_size),
        albumentations.Normalize(p=1.0),
        ToTensorV2()
    ])



class FGVC(Dataset):
    def __init__(self,df,mode='train',transforms=None):
        self.transforms = transforms
        self.dataf = df
        self.mode = mode
    def __len__(self):
        return len(self.dataf)
    def __getitem__(self,idx):
        img_name = self.dataf.loc[idx,'image']
        if self.mode == 'train':
            img = cv2.imread(os.path.join(cfg.train_root_dir,img_name))
        elif self.mode =='val':
            img = cv2.imread(os.path.join(cfg.val_root_dir,img_name))
        #print(img_name)
        label = self.dataf.iloc[idx][cfg.classes]
        if self.transforms is not None:
            img = self.transforms(image=img)['image']
        return img,torch.tensor(label)


In [32]:
mis = Mish()

In [33]:
class FGVCNet(pl.LightningModule):
    def __init__(self):
        super(FGVCNet,self).__init__()

        if "resnet" in cfg.model_name:
            self.model = eval(cfg.model_name)(pretrained=False)
            for params in self.model.parameters():
                params.require_grad = True
            self.model = nn.Sequential(*list(self.model.children())[:-2])
            self.model.add_module("average_pool",nn.AdaptiveAvgPool2d(output_size=(1,1)))
            self.model.add_module("Flatten",nn.Flatten())
            self.model.add_module("FC1",nn.Linear(2048,1024))
            self.model.add_module("ACT1",mis)
            self.model.add_module("FC2",nn.Linear(1024,1024))
            self.model.add_module("ACT2",mis)
            self.model.add_module("CLASSIFIER",nn.Linear(1024,5))

        if "efficient" in cfg.model_name:
            self.model = timm.create_model(cfg.model_name,pretrained=True)
            num_ftrs = self.model.classifier.in_features
            self.model.classifier = nn.Linear(num_ftrs,5)
        

    def forward(self,x):
        return self.model(x)
    
    def configure_optimizers(self):
        op = opt.Ranger(self.parameters(),lr=0.01)
        
        scheduler = {
            'scheduler':optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer=op,T_0=10),
            'monitor':'val_f1',
            'interval':'epoch',
            'frequency':1,
            'strict':True
        }

        self.op = op
        self.scheduler = scheduler
        
        return [op],[scheduler]

    def training_step(self,batch,batch_idx):
        X,y = batch
        y_hat = self.model(X)
        loss_tr = F.binary_cross_entropy_with_logits(y_hat,y.float())
        f1_tr = pl.metrics.functional.f1(y_hat.sigmoid(),y,5) 
        self.log("TrainLoss",loss_tr,prog_bar=True,on_step=False,on_epoch=True,logger=True)
        self.log("TrainF1",f1_tr,prog_bar=True,on_epoch=True,on_step=False,logger=True)
        return loss_tr
    
    def validation_step(self,batch,batch_idx):
        X,y = batch
        y_hat = self.model(X)
        loss_val = F.binary_cross_entropy_with_logits(y_hat,y.float())
        f1_val = pl.metrics.functional.f1(y_hat.sigmoid(),y,5)
        self.log("val_loss",loss_val,prog_bar=True,on_step=False,on_epoch=True,logger=True)
        self.log("val_f1",f1_val,prog_bar=True,on_epoch=True,on_step=False,logger=True)
        return loss_val

In [34]:
train_data = pd.read_csv("train2.csv")

In [35]:
def main():
    
    i=0

    comet = CometLogger(api_key="TFI1zlhr3wXWanvKsvRrDaGc4",workspace="thirurjst",save_dir='comet',project_name="fgvc8",experiment_name=f"{cfg.model_name}_{cfg.img_size}_Model Fold - {i+1}/5"+"\t"+"Time:"+str(time.time()))

    estp = EarlyStopping(monitor="val_f1",mode="max",patience=cfg.patience[0])

    if os.path.isdir(f"{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}_2"):
        print("Already Exists")
    else:
        os.makedirs(f"{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}_2")
    

    mod_ckpt = ModelCheckpoint(
    monitor='val_loss',
    mode='min',
    dirpath=f"{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}_2",
    filename='Resnet18_512_Checkpoint-ValLoss:{val_loss:.4f}-F1:{val_f1:.4f}',
    save_top_k=1,
)
    trainer = pl.Trainer(gpus=1,precision=16,max_epochs=cfg.max_epochs,progress_bar_refresh_rate=30,deterministic=True,benchmark=True,callbacks=[mod_ckpt,estp],logger=comet,accumulate_grad_batches=1)
    
    
    print(f"Initializing model Fold - {i+1}/5")
    model = FGVCNet()
    print("*** Model Initialization Completed ***")
    train_recs = train_data[train_data.folds != i].reset_index(drop=True)
    val_recs = train_data[train_data.folds == i].reset_index(drop=True)

    train_dataset = FGVC(df=train_recs,mode='train')
    val_dataset = FGVC(df=val_recs,mode='val',transforms=val_augs())

    train_loader = DataLoader(train_dataset,batch_size=32,num_workers=4)
    val_loader = DataLoader(val_dataset,batch_size=32,num_workers=4)

    trainer.fit(model,train_loader,val_loader)

    print("Uploading Model to Comet")
    comet.experiment.log_model(f"Model{i+1}",f"./{cfg.model_name}_{cfg.img_size}_Model_Fold-{i+1}_2")
    print("Upload OK...")
    comet.experiment.end()


In [36]:
main()

CometLogger will be initialized in online mode
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.
Already Exists
Initializing model Fold - 1/5
COMET INFO: ---------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.ml/thirurjst/fgvc8/334844889c914f2aa3cb4bbac781b74e
COMET INFO:   Others:
COMET INFO:     Name : resnet50_512_Model Fold - 1/5	Time:1619071810.946502
COMET INFO:   Uploads:
COMET INFO:     environment details      : 1
COMET INFO:     filename                 : 1
COMET INFO:     git metadata             : 1
COMET INFO:     git-patch (uncompressed) : 1 (137 KB)
COMET INFO:     installed packages       : 1
COMET INFO:     os packages              : 1
COMET INFO: ---------------------------
*** Model Initialization Complet

TypeError: __init__() takes 1 positional argument but 2 were given